In [ ]:
'''Relevant tutorials:
https://engineering.quora.com/Semantic-Question-Matching-with-Deep-Learning
https://explosion.ai/blog/quora-deep-text-pair-classification
'''

In [3]:
import tensorflow as tf
import keras
import pandas as pd
import spacy
from sklearn.model_selection import train_test_split
import pandas as pd
import csv
import os
import numpy as np
import pickle
from time import time
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from copy import deepcopy
stop_words = set(stopwords.words("english"))
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import Merge
from keras.models import Model
from keras.layers import Embedding, Input
import keras.backend as K
from keras.layers import LSTM, Bidirectional
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import Adadelta

Using TensorFlow backend.


In [4]:
time()

1510393783.97955

In [5]:
df_train = pd.read_csv("train.csv")

In [6]:
df_train

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [7]:
def tokenize_question(q, lower= False):
    '''Either use NLTK for this or use: https://github.com/myleott/ark-twokenize-py/blob/master/twokenize.py '''
    try:
        tokens = word_tokenize(q)
    except UnicodeDecodeError:
        tokens = word_tokenize(q.decode('utf-8'))
    except:
        return ["<UNK>"]
    word_tokens = [word for word in tokens if word.isalpha()]  #only include words; not sure if best option
    word_tokens = [word for word in word_tokens if word not in stop_words]
    if(lower):
        word_tokens = map(lambda x: x.lower(), word_tokens) #converting all to lower case
    return word_tokens

In [8]:
%less

UsageError: Missing filename.


In [9]:
#All initial values, running this cell will reset all base variables
word_to_int = {"<UNK>":0}
int_to_word = {0:"<UNK>"}
X_training  = df_train.as_matrix()

In [10]:
vectors_dir = "/home/anna/gensim_format_glove.6B.300d.txt"
from gensim.models import KeyedVectors 
#Hyperparameters and initial settings
embedding_dims = 300
hidden_dims = 64
gradient_clipping_norm = 1.25
n_epoch = 25
n_batch = 64

In [11]:
df_train

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0


In [12]:

i=0 

#def add_to_vocab(token):
 #   word_to_int[token] = len(word_to_int)
  #  int_to_word[len(word_to_int)] = token
'''Calculate max sequence length for padding'''
seq_length_list = []

def get_word_to_int_sequence(tokens):
    '''Returns sequence and updates vocab'''
    '''Does increasing number of functions impact performance?'''
    seq = []
    #global max_seq_length
    for token in tokens:
        if(token not in word_to_int):
            word_to_int[token] = len(word_to_int)
            int_to_word[len(word_to_int)] = token
            seq.append(word_to_int[token])
        else:
            seq.append(word_to_int[token])
    #if(len(seq)>max_seq_length):
     #   max_seq_length = len(seq)
    seq_length_list.append(len(seq))
    return seq
'''
for row in X_training:
    
    i+=1
    if(i==5):
        break
    q1, q2 = row[3],row[4]
    q1_tokens, q2_tokens = tokenize_question(q1),tokenize_question(q2)
    q1_seq, q2_seq = get_word_to_int_sequence(q1_tokens), get_word_to_int_sequence(q2_tokens)
    print(q1_seq, q2_seq)
    print(q1, q2)
    '''

'\nfor row in X_training:\n    \n    i+=1\n    if(i==5):\n        break\n    q1, q2 = row[3],row[4]\n    q1_tokens, q2_tokens = tokenize_question(q1),tokenize_question(q2)\n    q1_seq, q2_seq = get_word_to_int_sequence(q1_tokens), get_word_to_int_sequence(q2_tokens)\n    print(q1_seq, q2_seq)\n    print(q1, q2)\n    '

In [13]:
''' This cell does the following:
Through each row of given training set:
1) Update vocab (word_to_int) and inverse
2) Create training sequences '''

def process_dataset(dataset, dodeepcopy = True):
    '''Input is a numpy array of questions
    Output is an array of sequences according to word_to_int dict'''
    if(not dodeepcopy):
        dataset_mod = dataset
    else:
        dataset_mod = deepcopy(dataset)
    
    for i, row in enumerate(dataset_mod):
        #print(i) #for debugging
        q1,q2 = row[3],row[4]  #these correspond to the question
        q1_tokens, q2_tokens = tokenize_question(q1),tokenize_question(q2)
        q1_seq, q2_seq = get_word_to_int_sequence(q1_tokens), get_word_to_int_sequence(q2_tokens)
        row[3],row[4] = q1_seq, q2_seq
        
    
    return dataset_mod

X_training = process_dataset(X_training, dodeepcopy= False)

In [14]:
X_training[105781][3]
X_training.shape
X_training[1]

array([1, 3, 4, list([1, 8, 9, 10]),
       list([1, 11, 12, 13, 14, 15, 9, 16, 17]), 0], dtype=object)

In [20]:
max_seq_length = round(np.mean(seq_length_list) + 2*np.std(seq_length_list))

In [15]:
tokenize_question("""On Snapchat, someone blocked me, but still shows up in my added me section, but Snapchat still won't allow me to add them. What\xe2\x80\x99s going on?""")

[u'On',
 u'Snapchat',
 u'someone',
 u'blocked',
 u'still',
 u'shows',
 u'added',
 u'section',
 u'Snapchat',
 u'still',
 u'wo',
 u'allow',
 u'add',
 u'What',
 u'going']

In [63]:
tokenize_question(np.nan)

['<UNK>']

In [14]:
'''Load embeddings'''
word_vectors = KeyedVectors.load_word2vec_format(vectors_dir, binary=False)
word_embeddings = np.random.randn(len(word_to_int), embedding_dims)
word_embeddings[0] = 0
for word, i in word_to_int.items():
    if(word in word_vectors.vocab):
        word_embeddings[i] = word_vectors.word_vec(word)

In [79]:
validation_size = int(round((0.1)*X_training.shape[0]))
training_size = X_training.shape[0] - validation_size
print("Validation size:{} and training size:{}".format(validation_size, training_size))

Validation size:40429 and training size:363861


In [80]:
X_qs = np.array(map(lambda x:[x[3],x[4]], X_training))
Y = np.array(map(lambda x: [x[-1]], X_training))
X_train, X_val, Y_train, Y_val = train_test_split(X_qs, Y, test_size = validation_size)

In [84]:
X_train.shape
max_seq_length = max_seq_length

In [102]:
pad_seq = lambda x: sequence.pad_sequences(x,maxlen=max_seq_length)

# Split to dicts
X_training = {'left': pad_seq(X_train[:,0]), 'right': pad_seq(X_train[:,1])}
X_validation = {'left': pad_seq(X_val[:,0]), 'right': pad_seq(X_val[:,1])}


In [127]:
'''Final model'''

def l1_dist_exp(left, right):
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True)) #returns negative l1 norm exponent across batches

q1_placeholder = Input(shape=(max_seq_length,), dtype ="int32")
q2_placeholder = Input(shape=(max_seq_length,), dtype ="int32")
embed_layer = Embedding(len(word_embeddings), embedding_dims, weights = [word_embeddings], input_length = max_seq_length, trainable = False)
#Experiment with trainable but large corpus should not affect

embedded_q1, embedded_q2 = embed_layer(q1_placeholder), embed_layer(q2_placeholder)

siamese_LSTM = LSTM(hidden_dims)
q1_enc, q2_enc = siamese_LSTM(embedded_q1), siamese_LSTM(embedded_q2)

l1_dist = Merge(mode = lambda x: l1_dist_exp(x[0],x[1]), output_shape = lambda x: (x[0][0],1))([q1_enc, q2_enc])

final_model = Model(inputs = [q1_placeholder, q2_placeholder], outputs = [l1_dist]) 

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:16: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [128]:
optimizer = Adadelta(clipnorm = gradient_clipping_norm)
final_model.compile(loss = "mean_squared_error", optimizer = optimizer, metrics= ['accuracy'])

In [ ]:
t1 = time()
trained_model = final_model.fit([X_training['left'], X_training['right']], Y_train, batch_size = n_batch, nb_epoch= n_epoch, validation_data = ([X_validation['left'], X_validation['right']], Y_val))
print("Training finished.\n {}epochs in {}".format(n_epoch, time()-t1))

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app


Train on 363861 samples, validate on 40429 samples
Epoch 1/25
  7936/363861 [..............................] - ETA: 1899s - loss: 0.3236 - acc: 0.6473 

In [ ]:
'''Final model'''

def l1_dist_exp(left, right):
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True)) #returns negative l1 norm exponent across batches

q1_placeholder = Input(shape=(max_seq_length,), dtype ="int32")
q2_placeholder = Input(shape=(max_seq_length,), dtype ="int32")
embed_layer = Embedding(len(word_embeddings), embedding_dims, weights = [word_embeddings], input_length = max_seq_length, trainable = False)
#Experiment with trainable but large corpus should not affect

embedded_q1, embedded_q2 = embed_layer(q1_placeholder), embed_layer(q2_placeholder)

siamese_LSTM = Bidirectional(LSTM(hidden_dims))
q1_enc, q2_enc = siamese_LSTM(embedded_q1), siamese_LSTM(embedded_q2)

l1_dist = Merge(mode = lambda x: l1_dist_exp(x[0],x[1]), output_shape = lambda x: (x[0][0],1))([q1_enc, q2_enc])

final_model = Model(inputs = [q1_placeholder, q2_placeholder], outputs = [l1_dist]) 